In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [12]:
import random


def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # id товара
    items = np.array(items_weights['item_id'])
    # вес товара
    weights = np.array(items_weights['weight'])
    recommend = random.choices(items, weights = weights, k = n)

    return recommend

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [10]:
%%time
# суммирование чеков по одинаковым товарам у 1 пользователя
df_items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
# Расчитать вес как отношение данного чека к сумме всех
df_items_weights['weight'] = df_items_weights['sales_value']/df_items_weights['sales_value'].sum()
df_items_weights.drop(['sales_value'], axis = 1, inplace = True)
df_items_weights.head(3)

CPU times: total: 250 ms
Wall time: 227 ms


,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07
2,26093,2.254623e-07


In [13]:
result['weight_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(df_items_weights, n=5))
result.head(2)

,user_id,actual,weight_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[899624, 969866, 996269, 1003836, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[908064, 12132398, 846907, 849505, 1623493]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[13776783, 924705, 6602715, 1573870, 13115428]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[852452, 958214, 9221970, 1042199, 6553211]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [23]:
# your_code
result.reset_index()
result.actual.iloc[1]

'[  835476   851057   872021   878302   879948   909638   913202   920626\n   958154   994891  1053690  1083328  1096727  6463658  7167218  7167249\n  9526563  9526886 13842214]'

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
# your_code